# Домашнее задание # 3

## Задание 1

1. Будет решаться задача классификации
2. Целевые значения 1 - позитивный отзыв, 0 - негативный. Изначально в данных содержится оценка, которую пользователь поставил банку. Предполагается, что сама оценка в большей степени зависит от характеристик пользователя, а не от содержания отзыва, поэтому необходимо преобразовать оценки в бинарную переменную, характеризующую тональность отзыва. Отзывы с оценками 4 и 5 являются позитивными, отзывы с оценками 1, 2 и 3 - негативными. Прогнозы будут так же являться 1 или 0 в зависимости от предсказанной тональности озыва.
3. Для того, чтобы оценить качество модели, будут использоваться такие показатели, как recall и precision
4. Демонстрация должна работать на отзывах на русском языке длиной свыше 500 символов (минимальное ограничение на длину отзыва на banki.ru). Перед запуском модели нужно попытаться "вычистить" данные от ошибок и сленговых слов, однако избавиться от них на 100% скорее всего не получится, это также необходимо учесть в модели.

## Задание 2

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
import scrapy

class BanksSpider(scrapy.Spider):
    name = 'banks'
    
    def __init__(self):
        self.domain = 'http://www.banki.ru/' 
        self.download_delay = 3. 
        
    def start_requests(self):
        for page in range(1, 4): # сделаем тестовый парсинг только с 3 страниц
            yield scrapy.Request("http://www.banki.ru/services/responses/list/?page={}".format(page))
            
    def parse(self, response):
        for href in response.css('div.responses__item__message a::attr(href)'):
            yield response.follow(href, self.parse_review)
            
    def parse_review(self, response):
        bank_text = response.xpath('//div[@itemprop="name"]/text()').extract()
        bank = [p.strip() for p in bank_text[0].split('\t') if p != ''][0]
        
        rating_text = response.xpath('//span[@itemtype="http://data-vocabulary.org/Rating"]/text()').extract()
        
        if len(rating_text) == 0:
            rating = np.nan 
        else:
            rating = float([p.strip() for p in rating_text[1].split('\t') if p != ''][1])
            
        topic_text = response.xpath('//h0[@itemprop="summary"]/text()').extract()
        topic = [p.strip() for p in topic_text[0].split('\t') if p != ''][1]
    
        review_text = [' '.join(response.xpath('//div[@itemprop="description"]/text()').extract())]
        review = [p.strip() for p in review_text[0].split('\t') if p != ''][1]
        review = [' '.join([p.strip() for p in review.split('\xa0') if p != ''])][0]

        date_text = response.xpath('//time[@itemprop="dtreviewed"]/text()').extract()
        date = date_text[0]
        
        yield {
        'bank' : bank,
        'rating' : rating,
        'topic' : topic,
        'review' : review,
        'date' : date
        }

In [3]:
%%time
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor
from scrapy.utils.log import configure_logging

configure_logging()
runner = CrawlerRunner({'FEED_FORMAT': 'json', 'FEED_URI': 'banks_scrapy.json', 'LOG_LEVEL': 'INFO', 
                        'USER_AGENT': 'my-cool-project (http://example.com)'})
d = runner.crawl(BanksSpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

2018-02-12 15:53:22 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'banks_scrapy.json', 'LOG_LEVEL': 'INFO', 'USER_AGENT': 'my-cool-project (http://example.com)'}
2018-02-12 15:53:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2018-02-12 15:53:24 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermidd

Wall time: 4min 53s


### Пример данных

In [4]:
import json
with open('C:/Users/anastasiia.verbitska/Documents/GitHub/ml2018jan_feb/seminar08/banks_scrapy.json', 'r') as f:
    example = json.load(f)
example

[{'bank': 'Сбербанк России',
  'date': ' 12.02.2018 15:51',
  'rating': 1.0,
  'review': 'Подавала заявление в Сбербанк на снижение процентной ставки по ипотечному кредиту. Через месяц ответили отказом ссылаясь на реструктуризацию кредита. Очень жаль, что, когда я принимала участие в программе помощи ипотечным заемщикам от АИЖК, со стороны Сбербанка не было предупреждения какие для меня могут быть последствия и к тому же меня не предупредили, что это "реструктуризация". В жизни бывают разные ситуации, но совершенно непонятно такое разделение. Теперь же, благодаря этой программе, я получаю отказ в рефинансировании от всех банков.',
  'topic': 'Отказ в рефинансировании'},
 {'bank': 'ДельтаКредит',
  'date': ' 12.02.2018 15:25',
  'rating': 5.0,
  'review': 'Добрый день. Я являюсь клиентом банка "ДельтаКредит" Хочу выразить свою благодарность ведущему специалисту по ипотечному кредитованию, Петровскому Артему,который готовил и сопровождал сделку.Скорость и качество обслуживания, на высшем